In [77]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [78]:
currency = pd.read_excel('C:/Users/lenovo/Desktop/EURUSD.xlsx') # read the excel file
currency

,Date,Open,High,Low,Close
0,2022-06-14 12:36:00,1.0420,1.0420,1.0419,1.0419
1,2022-06-14 12:35:00,1.0421,1.0422,1.0419,1.0419
2,2022-06-14 12:34:00,1.0423,1.0423,1.0421,1.0421
3,2022-06-14 12:33:00,1.0422,1.0423,1.0421,1.0423
4,2022-06-14 12:32:00,1.0422,1.0422,1.0420,1.0422
...,...,...,...,...,...
4109,2022-06-09 17:04:00,1.0617,1.0618,1.0617,1.0618
4110,2022-06-09 17:03:00,1.0617,1.0617,1.0617,1.0617
4111,2022-06-09 17:02:00,1.0617,1.0617,1.0617,1.0617
4112,2022-06-09 17:01:00,1.0617,1.0617,1.0616,1.0617


In [79]:
# Modify the form
currency['return'] = np.log(currency['Close']) - np.log(currency['Open'])
del currency['Open'], currency['Close']
cur_r= currency[['Date','return']]
cur_r

,Date,return
0,2022-06-14 12:36:00,-0.000096
1,2022-06-14 12:35:00,-0.000192
2,2022-06-14 12:34:00,-0.000192
3,2022-06-14 12:33:00,0.000096
4,2022-06-14 12:32:00,0.000000
...,...,...
4109,2022-06-09 17:04:00,0.000094
4110,2022-06-09 17:03:00,0.000000
4111,2022-06-09 17:02:00,0.000000
4112,2022-06-09 17:01:00,0.000000


In [88]:
# Normalize the return value
r_min = cur_r['return'].min()
r_max = cur_r['return'].max()
cur_r['return'] = (cur_r['return'] - r_min) / (r_max - r_min)
cur_r

<ipython-input-88-166ac19e0563>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_r['return'] = (cur_r['return'] - r_min) / (r_max - r_min)


,Date,return
0,2022-06-14 12:36:00,0.730524
1,2022-06-14 12:35:00,0.708064
2,2022-06-14 12:34:00,0.708072
3,2022-06-14 12:33:00,0.775442
4,2022-06-14 12:32:00,0.752986
...,...,...
4109,2022-06-09 17:04:00,0.775030
4110,2022-06-09 17:03:00,0.752986
4111,2022-06-09 17:02:00,0.752986
4112,2022-06-09 17:01:00,0.752986


In [89]:
# Use return in last 5 minute as features, then generate features and label matrix
cur_r['r_{t-1}'] = cur_r['return'].shift(-1)
cur_r['r_{t-2}'] = cur_r['return'].shift(-2)
cur_r['r_{t-3}'] = cur_r['return'].shift(-3)
cur_r['r_{t-4}'] = cur_r['return'].shift(-4)
cur_r['r_{t-5}'] = cur_r['return'].shift(-5)
cur_r

<ipython-input-89-106910aebaa1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_r['r_{t-1}'] = cur_r['return'].shift(-1)


,Date,return,r_{t-1},r_{t-2},r_{t-3},r_{t-4},r_{t-5}
0,2022-06-14 12:36:00,0.730524,0.708064,0.708072,0.775442,0.752986,0.730530
1,2022-06-14 12:35:00,0.708064,0.708072,0.775442,0.752986,0.730530,0.752986
2,2022-06-14 12:34:00,0.708072,0.775442,0.752986,0.730530,0.752986,0.752986
3,2022-06-14 12:33:00,0.775442,0.752986,0.730530,0.752986,0.752986,0.820360
4,2022-06-14 12:32:00,0.752986,0.730530,0.752986,0.752986,0.820360,0.820379
...,...,...,...,...,...,...,...
4109,2022-06-09 17:04:00,0.775030,0.752986,0.752986,0.752986,0.752986,NaN
4110,2022-06-09 17:03:00,0.752986,0.752986,0.752986,0.752986,NaN,NaN
4111,2022-06-09 17:02:00,0.752986,0.752986,0.752986,NaN,NaN,NaN
4112,2022-06-09 17:01:00,0.752986,0.752986,NaN,NaN,NaN,NaN


In [90]:
# Use the first 700 rows as the testing set, and the rest as the training set
x_train = cur_r[['r_{t-1}','r_{t-2}','r_{t-3}','r_{t-4}','r_{t-5}']].iloc[700:-5]
x_train = x_train.to_numpy() # convert dataframe to ndarray
x_train

array([[0.73053895, 0.79788267, 0.77543764, 0.73053465, 0.77543764],
       [0.79788267, 0.77543764, 0.73053465, 0.77543764, 0.70808531],
       [0.77543764, 0.73053465, 0.77543764, 0.70808531, 0.75298614],
       ...,
       [0.70890145, 0.77502745, 0.77502952, 0.75298614, 0.75298614],
       [0.77502745, 0.77502952, 0.75298614, 0.75298614, 0.75298614],
       [0.77502952, 0.75298614, 0.75298614, 0.75298614, 0.75298614]])

In [91]:
x_train.shape

(3409, 5)

In [92]:
y_train = cur_r['return'].iloc[700:-5]
y_train = y_train.to_numpy()
y_train

array([0.70808531, 0.73053895, 0.79788267, ..., 0.79707083, 0.70890145,
       0.77502745])

In [102]:
# Build FNN
FNN_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(50, activation = 'relu', input_shape = (5, )),
    tf.keras.layers.Dense(50,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'linear')
])

In [103]:
# Train the model
FNN_model.compile(loss = "mean_squared_error")
FNN_model.fit(x_train, y_train, epochs = 100, verbose = 1)

Epoch 1/100
107/107 [==============================] - 0s 906us/step - loss: 0.0081
Epoch 2/100
107/107 [==============================] - 0s 895us/step - loss: 0.0031
Epoch 3/100
107/107 [==============================] - 0s 886us/step - loss: 0.0029
Epoch 4/100
107/107 [==============================] - 0s 892us/step - loss: 0.0027
Epoch 5/100
107/107 [==============================] - 0s 893us/step - loss: 0.0027
Epoch 6/100
107/107 [==============================] - 0s 884us/step - loss: 0.0026
Epoch 7/100
107/107 [==============================] - 0s 887us/step - loss: 0.0025
Epoch 8/100
107/107 [==============================] - 0s 972us/step - loss: 0.0024
Epoch 9/100
107/107 [==============================] - 0s 915us/step - loss: 0.0024
Epoch 10/100
107/107 [==============================] - 0s 883us/step - loss: 0.0023
Epoch 11/100
107/107 [==============================] - 0s 867us/step - loss: 0.0023
Epoch 12/100
107/107 [==============================] - 0s 893us/step - lo

107/107 [==============================] - 0s 906us/step - loss: 0.0017
Epoch 98/100
107/107 [==============================] - 0s 915us/step - loss: 0.0017
Epoch 99/100
107/107 [==============================] - 0s 894us/step - loss: 0.0016
Epoch 100/100
107/107 [==============================] - 0s 915us/step - loss: 0.0017


In [104]:
x_test = cur_r[['r_{t-1}','r_{t-2}','r_{t-3}','r_{t-4}','r_{t-5}']].iloc[:700]
x_test = x_test.to_numpy() # convert dataframe to ndarray
y_test = cur_r['return'].iloc[:700]
y_test = y_test.to_numpy()

In [105]:
FNN_model.predict(x_test)

22/22 [==============================] - 0s 810us/step


array([[0.76143414],
       [0.7613744 ],
       [0.76055694],
       [0.7585148 ],
       [0.76145476],
       [0.76551956],
       [0.7700419 ],
       [0.7652048 ],
       [0.7626891 ],
       [0.76266015],
       [0.7658435 ],
       [0.7548265 ],
       [0.753702  ],
       [0.7661697 ],
       [0.7615703 ],
       [0.75824255],
       [0.76670027],
       [0.76489276],
       [0.7670698 ],
       [0.7651707 ],
       [0.7575846 ],
       [0.75607383],
       [0.7644531 ],
       [0.75957805],
       [0.75826   ],
       [0.7607178 ],
       [0.76234215],
       [0.761416  ],
       [0.7602985 ],
       [0.7606515 ],
       [0.7647503 ],
       [0.76620626],
       [0.7611408 ],
       [0.7540856 ],
       [0.77251613],
       [0.76055604],
       [0.7573241 ],
       [0.76028985],
       [0.76567805],
       [0.77024895],
       [0.76638776],
       [0.7596628 ],
       [0.7557274 ],
       [0.7638051 ],
       [0.7700144 ],
       [0.7591382 ],
       [0.7548881 ],
       [0.762